# Symbolic reduction of given models
Passes a symbolic parameter and a data vector to the model function, then given the normal vector at the boundary (eigenvec with smallest EV of J'J), figures out a model with a reduce number of parameters

In [1]:
# Includes
using SymPy
using Iterators

## Subfunctions

### Utility

In [2]:
# Assign a vector of symbolic variables

# Assign a single variable, given it's string name and input
function string_as_varname(s::AbstractString,v::Any)
    s=symbol(s) 
    @eval (($s) = ($v))
end

# Assign a matrix of symbolic variables to string s (the variables share the same name)
function string_as_symb_vec(s::AbstractString, dims::Tuple)
    string_as_varname(s, [string_as_varname("$s$(i1)",Sym("$s$(i1)")) for i1 in 1:prod(dims)])
    string_as_varname(s, reshape(eval(symbol(s)), dims))
end

# Return the index of a symbolic variable within a vector

# Check if the input is already an array, if not, convert it to array
function arrayfy(a::Any)
    if (typeof(a)==Array{eltype(a),1}) || (typeof(a)==Array{eltype(a),2})
        return a # Already an array
    else
        return [a]
    end
end

arrayfy (generic function with 1 method)

In [145]:
# Return all combination of parameters given an integer vector
function abstract_mply(phi::AbstractArray, inds::AbstractArray)
    out = phi[inds[1]];
    for i1 in inds[2:end]
        out *= phi[i1]; 
    end
    return out
end

function all_phi_combs(phi::AbstractArray, v::AbstractArray; min_set_size=2)
    # Find parameters that need to be combined
    inds = find(x -> x != 0, v);
    n_comb = length(inds)
    phi = copy(phi)
    
    # Take the correct symbolic power before combining
    for i1 in inds[1:n_comb]
        phi[i1] = phi[i1].^Sym(round(Int, 1/abs(v[i1])))
    end
    
    # Also create both ^1 and ^-1 versions of each variable
    phi_ext = [phi; phi];
    for i1 = (1+length(phi)):(2*length(phi))
        phi_ext[i1] = 1/phi_ext[i1];
    end    
    
    out1 = Array(Any, ((2^length(inds))^2)); #Combinations
    out2 = Array(Any, ((2^length(inds))^2)); #Indices of generators
    
    # Iterate over all subsets and return them, ordered by their size   
    iterind = 0;
    for n1 = length(inds):-1:min_set_size
        for i1 in subsets(inds, n1)
            tmp = Array(Any, (n1,1))
            fill!(tmp, [0,length(phi)])
            for j1 in product(tmp...)
                #@show i1.+collect(j1)
                iterind += 1
                out1[iterind] = abstract_mply(phi_ext, i1.+collect(j1));
                out2[iterind] = i1.+collect(j1);
            end
        end
    end
    return out1[1:iterind], out2[1:iterind]
end

all_phi_combs (generic function with 1 method)

In [4]:
# Check if a combination of parameters is explained by other given combinations
function is_explained(expr::Sym, combs::AbstractArray)
    possible_combs, indices = all_phi_combs(combs, ones(size(combs)), min_set_size=1)
    explanation = find(x -> x == expr, possible_combs);
    
    # Returns whether or not it is explained, and if so, then also
    if !isempty(explanation)
        return !isempty(explanation), indices[explanation]
    else
        return !isempty(explanation)
    end
end

is_explained (generic function with 1 method)

### Important

In [147]:
# Given the normal at the boundary, substitute parameter combinations
function phi_combine(y_symb::Sym, v::AbstractArray)
    # First figure out what parameters need to be combined given v
    v[abs(v).<5e-2] = 0;
    v = round(v./abs(minimum(v[v.!=0])))
    # Note that this "integerization" of v persists outside of this subfunction, as we did NOT make a copy.
    
    # Get combinations, ordered by their size
    phi_comb, = all_phi_combs(phi, v)
    #@show phi_comb
    # Replace the combined phis with temporary etas
    y_comb = copy(y_symb);
    string_as_symb_vec("eta", (length(phi_comb),))
    for i1 in 1:length(phi_comb)
        y_comb = subs(y_comb, phi_comb[i1]=>eta[i1])
    end
    
    return y_comb, phi_comb, eta
end

phi_combine (generic function with 1 method)

In [30]:
# Take the limits of the combined model
function take_limits(y_comb::Sym, phi::AbstractArray, v::AbstractArray)
    inds = find(x -> x != 0, v); 
    y_lims = copy(y_comb)
    for i1 in inds
        if v[i1] > 0
            y_lims = limit(y_lims, phi[i1], oo)
        else
            y_lims = subs(y_lims, phi[i1], 0)
        end
    end
    
    return y_lims
end

take_limits (generic function with 1 method)

In [94]:
# Substitute back parameter combinations
function phi_reduce(y_lims::Sym, phi::AbstractArray, phi_comb::AbstractArray, eta::AbstractArray)
    y_red = copy(y_lims)
    # First check which combinations do exist in the reduced model, work with only those
    all_symbols = free_symbols(y_red);
    # We want to check which of the etas and phis we ended up using
    eta_does_exist = [];
    for i1 in 1:length(eta)
        ind = find(x -> x==eta[i1], all_symbols)
        if !isempty(ind) eta_does_exist = [eta_does_exist, i1] end
    end
    phi_does_exist = [];
    for i1 in 1:length(phi)
        ind = find(x -> x==phi[i1], all_symbols)
        if !isempty(ind) phi_does_exist = [phi_does_exist, i1] end
    end
    
    
    # Whether some higher order combinations can be explained by a combination of lower ordered ones
    phi_keep = copy(phi_comb[eta_does_exist])
    to_keep = length(phi_keep); # Check if all phi_combs are explained by the kept combinations
    expl_by = Array(Any, size(phi_keep));
    
    
    all_explained = false;
    while !all_explained
        all_explained = true;
        # Try to find an explanation for all combinations, given a few combinations
        for i1 in 1:length(phi_keep)
            b, tmp = is_explained(phi_keep[i1], arrayfy(phi_keep[to_keep]))
            expl_by[i1] = tmp[1]
            if !b all_explained=false end
        end
        
        # If there are some not explained, extend the set of explaining combinations with the smallest unexplained combination
        if !all_explained
            to_keep = [to_keep, find(x -> !isempty(x), expl_by[sort(collect(setdiff(Set(1:length(phi_keep)), Set(to_keep))))])[1]];
        end
    
    end
    
    # By now we found the minimal set of combinations that explain all combinations that exist in y_lims
    # Now replace the explained-away etas with combinations of to_keep etas
    for i1 = 1:length(phi_keep)
        y_red = subs(y_red, eta[eta_does_exist[to_keep[i1]]] =>  abstract_mply(eta, eta_does_exist[expl_by[i1]]));
    end
    
    
    # Now just create a reduced phi vector
    
    phi_red = copy(phi)[1:(length(to_keep)+length(phi_does_exist))]
    
    # Change the symbols within y_lims accordingly, and remember the mapping
    
    phi_red_mapping = Dict()
    # First map existing symbols to something arbitrary (kappa) and remember the mappings
    string_as_symb_vec("kappa", size(phi_red))
    for i1 in 1:length(phi_does_exist)
        y_red = subs(y_red, phi[phi_does_exist[i1]]=>kappa[i1])
        phi_red_mapping[phi[phi_does_exist[i1]]] = phi[i1];
    end
    for i1 in 1:length(to_keep)
        y_red = subs(y_red, eta[eta_does_exist[to_keep[i1]]]=>kappa[length(phi_does_exist)+i1])
        phi_red_mapping[phi_comb[eta_does_exist[to_keep[i1]]]] = phi[length(phi_does_exist)+i1];
    end
    
    # Create the final product by substituting back to phi form
    for i1 in 1:length(kappa)
        y_red = subs(y_red, kappa[i1]=>phi_red[i1])
    end
    
    
    return y_red, phi_red, phi_red_mapping
end

phi_reduce (generic function with 1 method)

In [133]:
# Turn the symbolic model into a callable function
function create_reduced_model(y_red::Sym, phi_in::AbstractArray, x_in::AbstractArray)
    # Roughly, with correct wrappers such that model_red(phi, x)

    f_red = lambdify(y_red, [phi_in; x_in])
    
    model_red(phi::AbstractArray, x::AbstractArray) = f_red(phi..., x...)
    
    return model_red
end

create_reduced_model (generic function with 1 method)

In [149]:
# Compute the reduced parameter vector numerically
function numeric_reduction(phi_boundary::AbstractArray, phi::AbstractArray, phi_red::AbstractArray, phi_red_mapping::Dict )
    # Quickly reverse the mapping for convenience
    phi_reverse_mapping = Dict();
    for key = keys(phi_red_mapping)
        phi_reverse_mapping[phi_red_mapping[key]] = key;
    end
    
    phi_red_numeric = zeros(size(phi_red))

    # Implement the numeric substitutions
    tmp = Array(Any, size(phi)) 
    for i1 in 1:length(phi)
        tmp[i1] = eval(symbol("phi$i1"))=>phi_boundary[i1];
    end
    
    # Do the substitutions
    for i1 in 1:length(phi_red)
        phi_red_numeric[i1] = subs(phi_reverse_mapping[eval(symbol("phi$i1"))], tmp...)
    end
    
    return phi_red_numeric
end

numeric_reduction (generic function with 2 methods)

## Main function

In [143]:
# Reduces the model described in the function model(phi::parameters, x::evaluation_locations)
# given a normal vector v of parameters at the boundary location x
# Return a callable function model_red(phi_red, x) where size(phi_red) = size(phi)-1
# but for the output y_red = model(phi_red, x):   (y_red-y)^2 ~ 0

function Reduce_model(model::Union{Function, Sym}, phi_boundary::AbstractArray, v_boundary::AbstractArray, data_dim::Tuple; model_id=0)
    # Create symbolic vectors for y and x
    string_as_symb_vec("phi", size(v_boundary)) # Dimension is number of parameters
    string_as_symb_vec("x", data_dim) # Dimension of input data dimension (for voltage and [Ca2+] is 2)
    
    # Get the symbolic model (either from a supplied function, or the symbolic version from previous iteration)
    if typeof(model)==Function
        #y_symb = model(phi, x, model_id=model_id) # General implementation
        y_symb = model(phi, Ca=[x[1]], V=[x[2]], model_id=model_id)[1] # BK_specific for now
    else
        y_symb = model;
    end

    # Given the normal at the boundary, substitute parameter combinations
    #y_comb, phi_comb = phi_combine(y_symb, v)
    y_comb, phi_comb, eta = phi_combine(y_symb, v)
    
    # Take the limits of the combined model
    y_lims = take_limits(y_comb, phi, v)
    
    # Substitute back parameter combinations
    y_red, phi_red, phi_red_mapping = phi_reduce(y_lims, phi, phi_comb, eta)
    
    # Turn the symbolic model into a callable function
    model_red = create_reduced_model(y_red, phi_red, x)
    
    # Compute the reduced parameter vector numerically
    phi_red_numeric = numeric_reduction(phi_boundary, phi, phi_red, phi_red_mapping)
    
    #return y_red, phi_red_numeric 
    
    return y_symb, y_red, phi_red_mapping, model_red, y_comb, phi_comb, y_lims, y_red, phi_red, phi_red_mapping, model_red, phi_red_numeric
end

Reduce_model (generic function with 1 method)

In [150]:
outp = Reduce_model(BK_simulator, exp(x_red2), v, (2,1), model_id=12);

In [151]:
outp[1]

                                                                              
                       /                                                      
                       |           38.1727608645031*phi2*x2   phi1*phi4*phi5*p
                       |phi1*phi5*e                         + ----------------
                       \                                                      
------------------------------------------------------------------------------
                                                                              
                                           4   /                              
/      38.1727608645031*phi2*x2        x1 \    |           38.1727608645031*ph
|phi1*e                         + 1 + ----|  + |phi1*phi5*e                   
\                                     phi3/    \                              

                                           4                        
        38.1727608645031*phi2*x2          \                  

In [152]:
outp[3]

Dict{Any,Any} with 5 entries:
  phi4*phi6 => phi5
  phi3      => phi3
  phi1      => phi1
  phi2      => phi2
  phi5      => phi4

In [153]:
outp[2]

                                                                              
                        /                                                     
                        |           38.1727608645031*phi2*x2   phi1*phi4*phi5*
                        |phi1*phi4*e                         + ---------------
                        \                                                     
------------------------------------------------------------------------------
                                                                              
/                                                         38.1727608645031*phi
|           38.1727608645031*phi2*x2   phi1*phi4*phi5*x1*e                    
|phi1*phi4*e                         + ---------------------------------------
\                                                          phi3               

                             4                       
    38.1727608645031*phi2*x2\                        
x1*e                  

In [154]:
outp[4]

model_red (generic function with 1 method)